In [2]:
# imports
import os
import numpy as np
import pandas as pd
import random as rn
import pickle as pkl
import tensorflow as tf
from keras_self_attention import SeqSelfAttention

# Loading the data

In [3]:
# import all the data from the general sentence embeddings
with open('data/sentence_embeddings/general/unsorted/sentemb/sentemb_unlabeled3.p', 'rb') as f:
    data_general = pkl.load(f)

with open('data/sentence_embeddings/general/unsorted/label_domain/label_domain_train_sentemb_unlabeled3.p', 'rb') as f:
    temp_train = pkl.load(f)

with open('data/sentence_embeddings/general/unsorted/label_domain/label_domain_test_sentemb_unlabeled3.p', 'rb') as f:
    temp_test = pkl.load(f)
    
labels_general = np.hstack((temp_train, temp_test))

data_general = data_general.transpose()

# import all the specific sentence embedding data - here domain 0 was chosen
with open('data/sentence_embeddings/specific/sentemb/sentemb_unlabeled3_1.p', 'rb') as f:
    data_spec = pkl.load(f)
    
with open('data/sentence_embeddings/specific/label_domain/label_domain_train_sentemb_unlabeled3_1.p', 'rb') as f:
    temp_train = pkl.load(f)

with open('data/sentence_embeddings/specific/label_domain/label_domain_test_sentemb_unlabeled3_1.p', 'rb') as f:
    temp_test = pkl.load(f)
    
labels_spec = np.hstack((temp_train, temp_test))

data_spec = data_spec.transpose()
labels_spec = labels_spec[0,:]
labels_spec = labels_spec.transpose()

In [39]:
labels_spec_train[0,:]

array([0, 1, 0, ..., 1, 1, 1])

In [57]:
labels_general.shape

(3, 31753)

In [4]:
# split the data 70-10-20 (train-validation-test) - data was already shuffled before
X_train = data_general[:22227]
X_val = data_general[22227:25402]
X_test = data_general[25402:]



labels_gen_train = labels_general[:22227]
labels_gen_val = labels_general[22227:25402]
labels_gen_test = labels_general[25402:]

# save data
pkl.dump(np.vstack((X_val, X_test)), open("data/sentence_embeddings/general/sorted/val_test/vt_data4_10_1.p", "wb"))
pkl.dump(np.hstack((labels_general[:,22227:25402],labels_general[:,25402:])), open("data/sentence_embeddings/general/sorted/val_test/vt_labels4_10_1.p", "wb"))

pkl.dump(np.vstack((X_train)), open("data/sentence_embeddings/general/sorted/train/train_data4_10_1.p", "wb"))
pkl.dump(np.hstack((labels_general[:,:22227])), open("data/sentence_embeddings/general/sorted/train/train_labels4_10_1.p", "wb"))

In [5]:
# set the target domain
index_spec = 1


# importing the data for the general sentence embeddings, here corresponding data from domain 0 was chosen
with open('data/sentence_embeddings/general/sorted/train/train_data3_10_1.p', 'rb') as f:
    X_train_gen = pkl.load(f)

with open('data/sentence_embeddings/general/sorted/train/train_labels3_10_1.p', 'rb') as f:
    y_train = pkl.load(f)
    
with open('data/sentence_embeddings/general/sorted/val_test/vt_data3_10_1.p', 'rb') as f:
    X_val_test_spec = pkl.load(f)

with open('data/sentence_embeddings/general/sorted/val_test/vt_labels3_10_1.p', 'rb') as f:
    y_val_test = pkl.load(f)

labels_total = np.hstack((y_train[:,:22227], y_val_test))
X_train_gen, X_val_gen, X_test_gen = X_train_gen[:22227], X_train_gen[:25402], X_train_gen[25402:]
y_train, y_val, y_test = y_train[0,:22227], y_train[0,:25402], y_train[0,25402:]


# import the data from the specific sentence embeddings, here corresponding data from domain 0 was chosen
with open('data/sentence_embeddings/specific/sentemb/sentemb_unlabeled3_1.p', 'rb') as f:
    X_spec = pkl.load(f)
    
#X_train_spec, X_val_spec, X_test_spec = X_spec[:1400], X_spec[1400:1600], X_spec[1600:2000] 

import numpy as np
#X_spec=np.repeat(X_spec,repeats=3, axis=1)

X_train_spec, X_val_spec, X_test_spec = X_spec.transpose()[:1400], X_spec.transpose()[1400:1600], X_spec.transpose()[1600:]

In [9]:
# initialize the proposed classifier

INPUT_SIZE = 300
LATENT_SIZE = 300

# domain-general model parts
inp_gen = tf.keras.Input(shape=(1,INPUT_SIZE))
#inp_gen_att, attn_weights_gen = SeqSelfAttention(return_attention = True)(inp_gen)
#layer0 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(LATENT_SIZE, input_shape=(None,1,INPUT_SIZE), return_sequences=True))
inp_spec = tf.keras.Input(shape=(1,INPUT_SIZE))
merged = tf.keras.layers.Concatenate()([inp_gen, inp_spec])
merged = tf.keras.layers.Dense(300, activation='sigmoid')(merged)
#merged2 = tf.keras.layers.Dense(100, activation='sigmoid')(merged)
   # merged = tf.keras.layers.Dense(100, activation='sigmoid')(merged)
# drop out layer and dense layer
merged = tf.keras.layers.Dropout(.4)(merged)
merged = tf.keras.layers.Dense(1, activation='sigmoid')(merged)

    
classifier_gen = tf.keras.Model([inp_gen,inp_spec], merged)

#classifier_spec =  tf.keras.Model([inp_gen,inp_spec], merged2)

#inp_spec_att, attn_weights_spec = SeqSelfAttention(return_attention = True)(inp_spec)
#out_spec = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(LATENT_SIZE, input_shape=(None,1,INPUT_SIZE)))(inp_spec)

#out= inner_model(inp_spec)
# concatenate domain-general and domain-specific results
#merged = tf.keras.layers.Concatenate()([out_gen, out_spec])

# drop out layer and dense layer

#inner_model = tf.keras.Model([inp_gen], merged)
#classifier4 = tf.keras.Model([inp_spec], out)
#classifier4.trainable=False
#classifier.summary()

# Check that the weights of layer1 have not changed during training


In [10]:
# training the general model
classifier_gen.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="weights/classifier/classifier_without_al/standard_model/classifier_domain3_1.h5")
history = classifier_gen.fit([np.expand_dims(X_train_gen, 1)], y_train, epochs=30, validation_data = ([np.expand_dims(X_val_gen, 1)], y_val), callbacks = [checkpoint, es], batch_size=32)

# evaluating the general model
score = classifier_gen.evaluate([np.expand_dims(X_test_gen, 1)], y_test, verbose=0) 
print('Final accuracy score: '+str(score[1]))

Epoch 1/30


ValueError: in user code:

    /usr/local/lib/python3.6/dist-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.6/dist-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.6/dist-packages/keras/engine/training.py:835 run_step  **
        outputs = model.train_step(data)
    /usr/local/lib/python3.6/dist-pack

In [72]:
# training the specific model
classifier4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="weights/classifier/classifier_without_al/standard_model/classifier_domain2_13.h5")
history = classifier4.fit([np.expand_dims(X_train_spec, 1)], labels_spec_train, epochs=30, validation_data = ([np.expand_dims(X_val_spec, 1)], labels_spec_val), callbacks = [checkpoint, es], batch_size=32)

# evaluating the specific model
score = classifier4.evaluate([np.expand_dims(X_test_spec, 1)], labels_spec_test, verbose=0) 
print('Final accuracy score: '+str(score[1]))

Epoch 1/30
44/44 [==============================] - 7s 41ms/step - loss: 0.9084 - accuracy: 0.5664 - val_loss: 0.6314 - val_accuracy: 0.6800
Epoch 2/30
44/44 [==============================] - 0s 11ms/step - loss: 0.5960 - accuracy: 0.6857 - val_loss: 0.5434 - val_accuracy: 0.7500
Epoch 3/30
44/44 [==============================] - 0s 11ms/step - loss: 0.5281 - accuracy: 0.7536 - val_loss: 0.4840 - val_accuracy: 0.7700
Epoch 4/30
44/44 [==============================] - 0s 11ms/step - loss: 0.4913 - accuracy: 0.7714 - val_loss: 0.4512 - val_accuracy: 0.7900
Epoch 5/30
44/44 [==============================] - 0s 11ms/step - loss: 0.4538 - accuracy: 0.8043 - val_loss: 0.4416 - val_accuracy: 0.8000
Epoch 6/30
44/44 [==============================] - 0s 11ms/step - loss: 0.4365 - accuracy: 0.8014 - val_loss: 0.4325 - val_accuracy: 0.8000
Epoch 7/30
44/44 [==============================] - 0s 11ms/step - loss: 0.4153 - accuracy: 0.8086 - val_loss: 0.4262 - val_accuracy: 0.8050
Epoch 8/30
44